In [4]:
import sys
sys.path.insert(0, '/mnt/deliang-data/projects/CovidDepressionAnalysis/scripts/data_processing')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D

import pickle as pkl

from scipy import stats
from sklearn.decomposition import PCA
import seaborn as sns
from constants import *


from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split


import shap
import xgboost
from xgboost import cv



/mnt/deliang-data/.conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
class LongitudinalAnalysis():
    def __init__(self, shrink=True):
        with open('../output/v3_python/full_imputed.pickle', 'rb') as f:
            imputed = pkl.load(f)

        if shrink == True:
            self.data = imputed['data'][:,:,1:13]
            self.waves = [x for x in range(2,14)]
            self.weeks = [wave_weeks[x] for x in self.waves]
        else:
            self.data = imputed['data']
            self.waves = [x for x in range(1, imputed['data'].shape[2]+1)]
            self.weeks = [wave_weeks[x] for x in self.waves]
        self.columns = imputed['columns']
        self.ids = imputed['cvdids']

        self.index_bdi = imputed['columns'].index('BDI')
        self.index_sah = imputed['columns'].index('Mandatory_SAH')

        self.sah_variability = []

        for i in range(self.data.shape[0]):
            indexes = np.array(self.data[i, imputed['columns'].index('Mandatory_SAH'),:], dtype=bool)
            if ~((indexes == 1).all() | (indexes == 0).all()):
                self.sah_variability.append(True)

    def compare_before_after_sah(self, data):
        before_sah = []
        after_sah = []
        for i in range(data.shape[0]):
            for j in range(data.shape[2]):
                if data[i, self.index_sah, j] == 1:
                    after_sah.append(data[i, self.index_bdi, j])
                else:
                    before_sah.append(data[i, self.index_bdi, j])
        return before_sah, after_sah
    
    def pearson_corr(self, x1, x2):
        return (stats.ttest_ind(x1, x2))
    
    def corr_m(self, m1, m2):
        corr_out = []
        pval_out = []

        for i in range(self.data.shape[0]):
            pearson_corr = stats.pearsonr(m1[i,:], m2[i,:])
            corr_out.append(pearson_corr[0])
            pval_out.append(pearson_corr[1])

        df = pd.DataFrame({'corr':corr_out, 'pval':pval_out})
        return df
    
    def corr(self, var1, var2):
        corr_out = []
        pval_out = []

        index_var1 = self.columns.index(var1)
        index_var2 = self.columns.index(var2)

        for i in range(self.data.shape[0]):
            m_var1 = self.data[i, index_var1, :]
            m_var2 = self.data[i, index_var2, :]
            pearson_corr = stats.pearsonr(m_var1, m_var2)
            corr_out.append(pearson_corr[0])
            pval_out.append(pearson_corr[1])

        df = pd.DataFrame({'corr':corr_out, 'pval':pval_out})
        return df
    
    def bdi_sah_corr(self):
        corr_out = []
        pval_out = []
        for i in range(self.data.shape[0]):
            BDI = self.data[i, self.index_bdi, :]
            SAH = self.data[i, self.index_sah, :]
            pearson_corr = stats.pearsonr(BDI, SAH)
            corr_out.append(pearson_corr[0])
            pval_out.append(pearson_corr[1])

        df = pd.DataFrame({'corr':corr_out, 'pval':pval_out})
        return df
    
    def extract_significant_subjects(self, df):
        significant_positive_indexes = ((df['pval'] < 0.05) & (df['corr'] > 0)).values
        significant_negative_indexes = ((df['pval'] < 0.05) & (df['corr'] < 0)).values
        
        neutral_indexes = (~df['corr'].isna()) & (~(significant_positive_indexes | significant_negative_indexes))
        return significant_positive_indexes, significant_negative_indexes, neutral_indexes
    

    def get_columns(self, columns):
        indexes = []
        for i in range(len(columns)):
            indexes.append(self.columns.index(columns[i]))

        return self.data[:, indexes,:]
    
    def plot_columns_with_groups(self, columns, groups):
        df = pd.DataFrame(columns=columns, data=np.mean(self.get_columns(columns), 2))
        df['group'] = groups

        df = df.melt(id_vars=['group'], value_vars=columns)

        sns.barplot(data=df, x='variable', y='value', hue='group')
        
    def subject_classification(self, positive_indexes, negative_indexes, neutral_indexes):
        cls = []
        for i in range(len(positive_indexes)):
            if positive_indexes[i] == True:
                cls.append('positive')
            elif negative_indexes[i] == True:
                cls.append('negative')
            elif neutral_indexes[i] == True:
                cls.append('neutral')
            else:
                cls.append('NA')
        return cls

    def plot_personality_3d(self, columns, c):

        personality = self.extract_columns(columns)
        print(personality.shape)
        x = np.mean(personality[:,0,:], axis=1)
        y = np.mean(personality[:,1,:], axis=1)
        z = np.mean(personality[:,2,:], axis=1)
        
        # Creating figure
        fig = plt.figure(figsize = (10, 7))
        ax = plt.axes(projection ="3d")
        
        print(personality[:,0].shape)

        # Creating plot
        ax.scatter(x, y, z, c=c)
        plt.title("simple 3D scatter plot")

        plt.legend()
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.set_zlim(0, 1)
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        
        # show plot
        plt.show()

    def detect_change(self, subset=None):

        if subset is None:
            data = self.data
        else:
            data = self.data[subset,:,:]

        SAH = data[:,self.index_sah,:]
        before_bdi = []
        after_bdi = []
        NEO_Neuroticism = []
        NEO_Extraversion = []
        NEO_Openness = [] 
        NEO_Agreeableness = [] 
        NEO_Conscientiousness = []

        for i in range(SAH.shape[0]):
            for j in range(SAH.shape[1]-1):
                if (SAH[i, j] == 1) & (SAH[i, j+1] == 0):
                    before_bdi.append(data[i, self.index_bdi, j])
                    after_bdi.append(data[i, self.index_bdi, j+1])
                    NEO_Neuroticism.append(data[i, self.columns.index('NEO_Neuroticism'), j])
                    NEO_Extraversion.append(data[i, self.columns.index('NEO_Extraversion'), j])
                    NEO_Openness.append(data[i, self.columns.index('NEO_Openness'), j])
                    NEO_Agreeableness.append(data[i, self.columns.index('NEO_Agreeableness'), j])
                    NEO_Conscientiousness.append(data[i, self.columns.index('NEO_Conscientiousness'), j])

        df = pd.DataFrame(columns=['before_sah_bdi','after_sah_bdi', 'NEO_Neuroticism', 'NEO_Extraversion', 'NEO_Openness', 'NEO_Agreeableness', 'NEO_Conscientiousness'], 
                          data=np.vstack([before_bdi, after_bdi, NEO_Neuroticism, NEO_Extraversion, NEO_Openness, NEO_Agreeableness, NEO_Conscientiousness]).T)

        return df
    
    def calculate_mean_different_before_after(self, col):
        SAH = self.data[:, self.index_sah, :]

        before_sah = []
        after_sah = []

        for i in range(SAH.shape[0]):

            before_indexes = SAH[i, :] == 1
            after_indexes = SAH[i, :] == 0
            if (np.sum(before_indexes) == len(before_indexes)) | (np.sum(before_indexes) == 0):
                before_sah.append(np.nan)
                after_sah.append(np.nan)

            else:
                before_sah.append(np.mean(self.data[i, self.columns.index(col), before_indexes]))
                after_sah.append(np.mean(self.data[i, self.columns.index(col), after_indexes]))

        return before_sah, after_sah
    
    def extract_data_right_before_after_sah(self, col):
        SAH = self.data[:, self.index_sah, :]

        before_sah = []
        after_sah = []

        for i in range(SAH.shape[0]):
            before_indexes = SAH[i, :] == 1
            after_indexes = SAH[i, :] == 0
            
            if (np.sum(before_indexes) == len(before_indexes)) | (np.sum(before_indexes) == 0):
                before_sah.append(np.nan)
                after_sah.append(np.nan)

            else:
                for j in range(SAH.shape[1]-1):
                    if SAH[i,j] != SAH[i,j-1]:
                        if SAH[i,j] == 1:
                            before_sah.append(self.data[i, self.columns.index(col), j])
                            after_sah.append(self.data[i, self.columns.index(col), j+1])
                        else:
                            before_sah.append(self.data[i, self.columns.index(col), j+1])
                            after_sah.append(self.data[i, self.columns.index(col), j])
                        break

        return before_sah, after_sah        
    

    def get_columns_before_after_sah(self, cols):
        dictionary = {}
        for col in cols:
            before_sah, after_sah = self.calculate_mean_different_before_after(col)
            dictionary[col+'_before'] = before_sah
            dictionary[col+'_after'] = after_sah

        return(pd.DataFrame(dictionary))

    def get_columns_righ_before_after_sah(self, cols):
        dictionary = {}
        for col in cols:
            before_sah, after_sah = self.extract_data_right_before_after_sah(col)
            dictionary[col+'_before'] = before_sah
            dictionary[col+'_after'] = after_sah

        return(pd.DataFrame(dictionary))

    
    def test_correlation(self, column):
        corrs = []
        pvals = []
        for i in range(self.data.shape[0]):
            index = self.columns.index(column)
            column_val = self.data[i, index, :]
            BDI = self.data[i, self.index_bdi, :]
            corr = stats.pearsonr(column_val, BDI)
            corrs.append(corr[0])
            pvals.append(corr[1])

        df = pd.DataFrame({'corr': corrs, 'pval': pvals})
        df['significance'] = df['pval'] < 0.05
        df['positive'] = df['corr'] > 0
        return df
    


In [6]:
analysis = LongitudinalAnalysis()

In [7]:
analysis.data.shape

(1177, 30, 12)

In [8]:
len(analysis.waves)

12

In [9]:
all_waves_df = []
for i in range(len(analysis.waves)):
    df = pd.DataFrame(analysis.data[:,:,i], columns=analysis.columns)
    df['wave'] = analysis.waves[i]
    all_waves_df.append(df)


In [10]:
long_form = pd.concat(all_waves_df).reset_index()

In [11]:
long_form.rename(columns={'index': 'Subject_ID'}, inplace=True)

In [12]:
long_form.to_csv('../output/v3_python/long_form.csv')

In [13]:
long_form

,Subject_ID,Race_A,Race_AA,Race_W,Gender,Political_Views,Age,Education,Income,Income_v2,...,deaths_avg,cases_avg_per_100k,deaths_avg_per_100k,slope_new_cases,slope_new_deaths,lat,lng,population,Mandatory_SAH,wave
0,0,0.0,0.0,1.0,0.0,0.083333,0.578125,0.500,0.4,0.571429,...,0.000000,0.001169,0.000000,0.546209,0.379953,0.486497,0.397570,0.004189,1.0,2
1,1,0.0,0.0,1.0,0.0,0.000000,0.718750,0.500,0.4,0.571429,...,0.000344,0.008680,0.004777,0.550796,0.383450,0.519114,0.767834,0.055232,1.0,2
2,2,0.0,0.0,0.0,1.0,0.500000,0.078125,0.000,0.0,0.000000,...,0.001394,0.010613,0.003185,0.546209,0.379953,0.297191,0.462466,0.328839,1.0,2
3,3,0.0,0.0,1.0,0.0,0.833333,0.453125,0.125,0.0,0.000000,...,0.001585,0.013447,0.006369,0.546209,0.379953,0.337163,0.468688,0.216511,1.0,2
4,4,0.0,0.0,1.0,0.0,0.500000,0.468750,0.250,0.0,0.285714,...,0.000000,0.002923,0.000000,0.545413,0.371795,0.588257,0.395998,0.010708,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14119,1172,0.0,0.0,1.0,0.0,0.583333,0.843750,0.500,0.1,0.357143,...,0.005525,0.061807,0.033510,0.545867,0.376387,0.403718,0.690642,0.133403,0.0,13
14120,1173,0.0,0.0,0.0,0.0,0.000000,0.218750,0.750,0.0,0.142857,...,0.000000,0.025769,0.000000,0.550948,0.385781,0.581192,0.417147,0.002026,0.0,13
14121,1174,0.0,0.0,1.0,1.0,0.000000,0.843750,0.625,0.0,0.285714,...,0.000134,0.018798,0.006369,0.542115,0.386946,0.493031,0.937298,0.017936,0.0,13
14122,1175,0.0,0.0,1.0,1.0,0.666667,0.312500,0.500,0.8,0.857143,...,0.002043,0.050009,0.020701,0.551478,0.375291,0.502879,0.972921,0.078908,0.0,13


In [35]:
bdi = analysis.data[10,analysis.index_bdi,:]
sah = analysis.data[10,analysis.index_sah,:]

In [36]:
bdi

array([0.06349206, 0.15873016, 0.15873016, 0.15873016, 0.25396825,
       0.19047619, 0.12698413, 0.15079365, 0.17460317, 0.15079365,
       0.12698413, 0.12698413])

In [37]:
sah

array([1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [38]:
stats.pearsonr(bdi, sah)

PearsonRResult(statistic=-0.30434782608695654, pvalue=0.3361433859161178)

In [31]:
long_form

,Subject_ID,Race_A,Race_AA,Race_W,Gender,Political_Views,Age,Education,Income,Income_v2,...,deaths_avg,cases_avg_per_100k,deaths_avg_per_100k,slope_new_cases,slope_new_deaths,lat,lng,population,Mandatory_SAH,wave
0,0,0.0,0.0,1.0,0.0,0.083333,0.578125,0.500,0.4,0.571429,...,0.000000,0.001169,0.000000,0.546209,0.379953,0.486497,0.397570,0.004189,1.0,2
1,1,0.0,0.0,1.0,0.0,0.000000,0.718750,0.500,0.4,0.571429,...,0.000344,0.008680,0.004777,0.550796,0.383450,0.519114,0.767834,0.055232,1.0,2
2,2,0.0,0.0,0.0,1.0,0.500000,0.078125,0.000,0.0,0.000000,...,0.001394,0.010613,0.003185,0.546209,0.379953,0.297191,0.462466,0.328839,1.0,2
3,3,0.0,0.0,1.0,0.0,0.833333,0.453125,0.125,0.0,0.000000,...,0.001585,0.013447,0.006369,0.546209,0.379953,0.337163,0.468688,0.216511,1.0,2
4,4,0.0,0.0,1.0,0.0,0.500000,0.468750,0.250,0.0,0.285714,...,0.000000,0.002923,0.000000,0.545413,0.371795,0.588257,0.395998,0.010708,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14119,1172,0.0,0.0,1.0,0.0,0.583333,0.843750,0.500,0.1,0.357143,...,0.005525,0.061807,0.033510,0.545867,0.376387,0.403718,0.690642,0.133403,0.0,13
14120,1173,0.0,0.0,0.0,0.0,0.000000,0.218750,0.750,0.0,0.142857,...,0.000000,0.025769,0.000000,0.550948,0.385781,0.581192,0.417147,0.002026,0.0,13
14121,1174,0.0,0.0,1.0,1.0,0.000000,0.843750,0.625,0.0,0.285714,...,0.000134,0.018798,0.006369,0.542115,0.386946,0.493031,0.937298,0.017936,0.0,13
14122,1175,0.0,0.0,1.0,1.0,0.666667,0.312500,0.500,0.8,0.857143,...,0.002043,0.050009,0.020701,0.551478,0.375291,0.502879,0.972921,0.078908,0.0,13


In [34]:
long_form.loc[long_form['Subject_ID']==10, 'BDI']

10       0.063492
1187     0.158730
2364     0.158730
3541     0.158730
4718     0.253968
5895     0.190476
7072     0.126984
8249     0.150794
9426     0.174603
10603    0.150794
11780    0.126984
12957    0.126984
Name: BDI, dtype: float64